## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pyarrow.feather as feather
from sklearn.model_selection import train_test_split 
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers.experimental import preprocessing


# W & B test

In [ ]:
import wandb
from wandb.keras import WandbCallback
!wandb login


## Prepare the data

In [ ]:
fluxData_df = feather.read_feather('data/fluxData.feather')
zernikeData_df = feather.read_feather('data/zernikeData.feather')
fluxData_df


In [ ]:
# Prepare the training dataset
X = fluxData_df
y = zernikeData_df
x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)


## Normalize data
as shown in https://keras.io/guides/preprocessing_layers/#normalizing-numerical-features 

In [ ]:
normalizer = preprocessing.Normalization()
a = normalizer.adapt(x_train)
x_train_norm = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0)
x_train_norm

# Build Model

In [ ]:
AoModel = keras.Sequential([
     keras.layers.InputLayer(19, name="digits"),
     keras.layers.Dense(2000, activation="relu"),
     keras.layers.Dense(1050, activation="relu"),
     keras.layers.Dense(100, activation="relu"),
     keras.layers.Dense(9, activation="linear", name="predictions"),

])
AoModel.summary()

## Compile and run model 

In [ ]:
batch_size = 128
epochs = 200


    # optimizer='sgd',
    # loss='mse',
    # metrics=[tf.keras.metrics.RootMeanSquaredError()])


opt = keras.optimizers.Adam(learning_rate=0.001)
AoModel.compile(loss= keras.losses.MeanSquaredError(),  optimizer=opt, metrics= [tf.keras.metrics.RootMeanSquaredError()])
AoModel.fit(x_train_norm, y_train, batch_size=batch_size, epochs=epochs,validation_split = 0.2, verbose = 2)

#AoModel.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[WandbCallback()])